In [10]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
### YOUR CODE HERE
from tensorflow.keras import regularizers
###
import tensorflow.keras.utils as ku 
import numpy as np
import keras
from keras import layers
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [11]:
tokenizer = Tokenizer()
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt \
    -O /tmp/sonnets.txt
data = open('/tmp/sonnets.txt').read()

corpus = data.lower().split("\n")


tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

--2021-02-10 17:19:51--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.217.128, 172.217.204.128, 74.125.141.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.217.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘/tmp/sonnets.txt’

/tmp/sonnets.txt    100%[===================>]  91.38K  --.-KB/s    in 0.002s  

2021-02-10 17:19:51 (58.8 MB/s) - ‘/tmp/sonnets.txt’ saved [93578/93578]



In [12]:
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

In [13]:

model = keras.models.Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(layers.Bidirectional(layers.LSTM(256, return_sequences=True)))
model.add(layers.Dense(total_words, activation='relu'))
model.add(layers.LSTM(128))
model.add(layers.Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics='accuracy')


In [14]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 10, 100)           321100    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 10, 512)           731136    
_________________________________________________________________
dense_2 (Dense)              (None, 10, 3211)          1647243   
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               1710080   
_________________________________________________________________
dense_3 (Dense)              (None, 3211)              414219    
Total params: 4,823,778
Trainable params: 4,823,778
Non-trainable params: 0
_________________________________________________________________


In [15]:
address="./kaggle/working/weights-improvement.hdf5"
stop = EarlyStopping(monitor = 'val_accuracy', min_delta = 0, 
                             patience = 5, verbose = 1, mode = 'auto')
save = ModelCheckpoint(address, monitor = 'val_accuracy', 
                               verbose = 0, save_best_only = True)
callbacks = [stop, save]

history = model.fit(predictors, label, validation_split=0.20, epochs=50, verbose=1, callbacks = callbacks)

Epoch 1/50
387/387 [==============================] - 7s 18ms/step - loss: 7.1098 - accuracy: 0.0236 - val_loss: 7.2111 - val_accuracy: 0.0165
Epoch 2/50
387/387 [==============================] - 6s 14ms/step - loss: 6.7363 - accuracy: 0.0227 - val_loss: 7.3822 - val_accuracy: 0.0223
Epoch 3/50
387/387 [==============================] - 5s 14ms/step - loss: 6.7297 - accuracy: 0.0222 - val_loss: 7.4998 - val_accuracy: 0.0165
Epoch 4/50
387/387 [==============================] - 6s 14ms/step - loss: 6.7298 - accuracy: 0.0213 - val_loss: 7.5696 - val_accuracy: 0.0210
Epoch 5/50
387/387 [==============================] - 5s 14ms/step - loss: 6.7306 - accuracy: 0.0223 - val_loss: 7.6251 - val_accuracy: 0.0162
Epoch 6/50
387/387 [==============================] - 6s 15ms/step - loss: 6.7250 - accuracy: 0.0231 - val_loss: 7.7134 - val_accuracy: 0.0162
Epoch 7/50
387/387 [==============================] - 5s 14ms/step - loss: 6.7308 - accuracy: 0.0225 - val_loss: 7.7333 - val_accuracy: 0.0162

Tried with val_accuracy, but still the training stopped as it did not see the improvement mentioned via min_delta = 0.